In [ ]:
import time
import sys

EMULATE_HX711=False     # HX711 에뮬레이터 사용여부(현재 사용하지 않음)

referenceUnit = -379          # 기준 저항값(1.25kg의 무게추를 기반으로 설정)

if not EMULATE_HX711:     # HX711 에뮬레이터를 사용하지 않을 때(GPIO 모듈 사용)
    import RPi.GPIO as GPIO
    from hx711 import HX711
else:                                 # HX711 에뮬레이터를 사용할 때
    from emulated_hx711 import HX711

def cleanAndExit():             # 프로그램 종료 프로세스
    print("Cleaning...")

    if not EMULATE_HX711:     # HX711 에뮬레이터를 사용하지 않을 때 GPIO모듈의 cleanup 함수 이용
        GPIO.cleanup()
        
    print("Bye!")
    sys.exit()

In [77]:
# 재고 변경 이력 기록 함수
# 파라미터(message) 타입 = str
import datetime
def write_log(message):
    log = open('inventory/weight_log.txt', mode = 'at', encoding = 'utf-8')
    log.write('\n------' + str(datetime.datetime.now()) + '------\n')
    log.write(message)
    log.write('\n------------end------------\n')
    log.close()

In [ ]:
# 재고 파일 불러오기
import pandas as pd
df = pd.read_csv("inventory/inventory_weight.csv")
df = df.set_index('상품명')

In [ ]:
# 저울 가동
hx = HX711(20,16)            # DT,SDK 핀이 연결된 GPIO 설정

hx.set_reading_format("MSB", "MSB")   # hx711의 데이터 읽기 형식 설정

hx.set_reference_unit(referenceUnit)      # hx711 모듈에 기준 저항값 적용

hx.reset()    # 모듈 초기화

hx.tare()      # 저울 기본 세팅(로드셀 위의 선반 무게 추출) 

print("Tare done! Add weight now...")

In [ ]:
pre_data = 0 # 이전 루틴의 물품 개수
while True:
    try:
        pre_data = df.loc['바나나우유'][0]
        val = round(hx.get_weight(5))   # 무게 측정 후 추출 (5는 측정 시 조정에 사용되는 값 개수, 작을수록 부정확&빠름)
        print(str(val)+'g')                     # 무게 출력

        hx.power_down()
        hx.power_up()
        time.sleep(0.1)
        
        if int(val/df.loc['바나나우유'][2]) != pre_data:   # 이전 루틴의 물품 개수가 현재 루틴의 물품개수와 다를 경우
            # 재고 정보 갱신
            df.loc['바나나우유'][0] = int(val/df.loc['바나나우유'][2])
            df.to_csv("inventory/inventory_weight.csv")
            
            # 재고 변경 이력 기록
            message = '상품명 : ' + '바나나우유' + '\n개수 변경 ( ' + str(predata) + ' -> ' + str(int(val/df.loc['바나나우유'][2])) + ' )'
            write_log(message)
        else:
            continue

    except (KeyboardInterrupt, SystemExit):  # 인터럽트 발생 시 프로그램 종료
        cleanAndExit()